In [29]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression

In [58]:
features = pd.read_csv('inforce2.csv')
features = features.iloc[:,2:]
features.head(5)

,gender,productType,age,ttm,gbAmt,gmwbBalance,withdrawal,FundValue1,FundValue2,FundValue3,FundValue4,FundValue5,FundValue6,FundValue7,FundValue8,FundValue9,FundValue10,fmv
0,F,ABRP,47.361644,19.265753,87657.368596,0.0,0.0,0.000000,0.000000,0.000000,45008.862226,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,16763.294834
1,M,ABRP,57.786301,18.180822,161534.095807,0.0,0.0,25681.176167,0.000000,23096.983282,16719.403927,19297.086334,25791.346577,24386.580008,0.000000,23949.686269,21906.863182,-4803.217415
2,M,ABRP,53.120548,18.180822,407190.045231,0.0,0.0,0.000000,0.000000,0.000000,0.000000,389147.884504,0.000000,0.000000,0.000000,0.000000,0.000000,-36038.487254
3,M,ABRP,42.863014,14.682192,307425.136759,0.0,0.0,0.000000,0.000000,52957.626980,0.000000,85110.564649,0.000000,0.000000,0.000000,62465.008957,0.000000,45419.126711
4,F,ABRP,62.709589,11.175342,356480.863596,0.0,0.0,0.000000,27505.372618,12784.848141,14425.316972,31306.967821,24655.367374,17119.720906,18851.816256,16424.639530,21829.421214,97013.359907


In [59]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)

In [60]:
features.head()

,age,ttm,gbAmt,gmwbBalance,withdrawal,FundValue1,FundValue2,FundValue3,FundValue4,FundValue5,...,productType_DBWB,productType_IBRP,productType_IBRU,productType_IBSU,productType_MBRP,productType_MBRU,productType_MBSU,productType_WBRP,productType_WBRU,productType_WBSU
0,47.361644,19.265753,87657.368596,0.0,0.0,0.000000,0.000000,0.000000,45008.862226,0.000000,...,0,0,0,0,0,0,0,0,0,0
1,57.786301,18.180822,161534.095807,0.0,0.0,25681.176167,0.000000,23096.983282,16719.403927,19297.086334,...,0,0,0,0,0,0,0,0,0,0
2,53.120548,18.180822,407190.045231,0.0,0.0,0.000000,0.000000,0.000000,0.000000,389147.884504,...,0,0,0,0,0,0,0,0,0,0
3,42.863014,14.682192,307425.136759,0.0,0.0,0.000000,0.000000,52957.626980,0.000000,85110.564649,...,0,0,0,0,0,0,0,0,0,0
4,62.709589,11.175342,356480.863596,0.0,0.0,0.000000,27505.372618,12784.848141,14425.316972,31306.967821,...,0,0,0,0,0,0,0,0,0,0


In [61]:
# Labels are the values we want to predict
labels = np.array(features['fmv'])

In [62]:
# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('fmv', axis = 1)

In [63]:
#dummy trap
features = features.drop(['gender_F', 'productType_WBSU'], axis=1)

In [64]:
# Saving feature names for later use
feature_list = list(features.columns)

In [65]:
# Convert to numpy array
features = np.array(features)

In [79]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25)

In [80]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (142500, 34)
Training Labels Shape: (142500,)
Testing Features Shape: (47500, 34)
Testing Labels Shape: (47500,)


In [81]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 10, random_state = 0)

In [82]:
# Train the model on training data
rf.fit(train_features, train_labels);

In [83]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

In [84]:
# Calculate the absolute errors
errors = abs(predictions - test_labels)

In [85]:
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 11413.72 degrees.


In [86]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

/home/novais/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/home/novais/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [87]:
# Calculate and display accuracy
accuracy = abs(100 - abs(np.mean(mape)))
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: nan %.


In [88]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: gbAmt                Importance: 0.31
Variable: productType_ABRU     Importance: 0.29
Variable: productType_IBRU     Importance: 0.16
Variable: productType_MBRU     Importance: 0.13
Variable: ttm                  Importance: 0.03
Variable: age                  Importance: 0.02
Variable: withdrawal           Importance: 0.01
Variable: productType_DBRP     Importance: 0.01
Variable: productType_DBRU     Importance: 0.01
Variable: productType_DBSU     Importance: 0.01
Variable: productType_IBSU     Importance: 0.01
Variable: gmwbBalance          Importance: 0.0
Variable: FundValue1           Importance: 0.0
Variable: FundValue2           Importance: 0.0
Variable: FundValue3           Importance: 0.0
Variable: FundValue4           Importance: 0.0
Variable: FundValue5           Importance: 0.0
Variable: FundValue6           Importance: 0.0
Variable: FundValue7           Importance: 0.0
Variable: FundValue8           Importance: 0.0
Variable: FundValue9           Importance: 0.0
Va

In [89]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 1000, random_state=42)
# Extract the two most important features
important_indices = [feature_list.index('productType_ABRU'),
                     feature_list.index('gbAmt'),
                     feature_list.index('productType_IBRU'),
                     feature_list.index('productType_MBRU'),
                     feature_list.index('productType_MBRU'),
                    feature_list.index('age'),
                    feature_list.index('ttm')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]

In [ ]:
# Train the random forest
rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')